#1. 데이터 읽기

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

moviefile = '/content/drive/MyDrive/6. 협업필터링/data/movies.csv'
ratingfile='/content/drive/MyDrive/6. 협업필터링/data/ratings.csv'
df_movies = pd.read_csv(moviefile)
df_ratings = pd.read_csv(ratingfile)

df_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


#2. 데이터 클린징

In [4]:
df_movies.info()
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [5]:
df_ratings.head()
df_ratings.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


> 이상치 및 결측치는 없는것으로 나타난다.

In [6]:
from datetime import datetime
for idx in range(5):
  print(datetime.fromtimestamp(df_ratings['timestamp'][idx]))

2000-07-30 18:45:03
2000-07-30 18:20:47
2000-07-30 18:37:04
2000-07-30 19:03:35
2000-07-30 18:48:51


In [7]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
df_ratings_movies = pd.merge(df_ratings, df_movies, on='movieId')
df_ratings_movies_pivot =  df_ratings_movies.pivot_table('rating', index='userId', columns='title')
df_ratings_movies_pivot

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [9]:
df_ratings_movies_pivot = df_ratings_movies_pivot.fillna(0)
df_ratings_movies_pivot

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

#3. 영화 간 유사도 계산하기

##3.1 사용자 기반 추천
> 위의 dataframe을 이용하여 사용자 1번과 성향이 가장 비슷한 순으로 10명을 출력하시오

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
user_sim = cosine_similarity(df_ratings_movies_pivot, df_ratings_movies_pivot)
df_user_sim = pd.DataFrame(data=user_sim, index=df_ratings_movies_pivot.index, columns=df_ratings_movies_pivot.index)
df_user_sim[1].sort_values(ascending=False)[1:11]

userId
266    0.357408
313    0.351562
368    0.345127
57     0.345034
91     0.334727
469    0.330664
39     0.329782
288    0.329700
452    0.328048
45     0.327922
Name: 1, dtype: float64

In [11]:
df_ratings_movies_pivot.loc[266].sort_values(ascending=False)[:10]

title
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    5.0
Crow, The (1994)                                                                  5.0
Goodfellas (1990)                                                                 5.0
Untouchables, The (1987)                                                          5.0
Dirty Work (1998)                                                                 5.0
Fear and Loathing in Las Vegas (1998)                                             5.0
Ref, The (1994)                                                                   5.0
Matrix, The (1999)                                                                5.0
This Is Spinal Tap (1984)                                                         5.0
Time Bandits (1981)                                                               5.0
Name: 266, dtype: float64

In [12]:
df_ratings_movies_pivot.loc[1].sort_values(ascending=False)[:10]

title
Messenger: The Story of Joan of Arc, The (1999)    5.0
Willy Wonka & the Chocolate Factory (1971)         5.0
Winnie the Pooh and the Blustery Day (1968)        5.0
Red Dawn (1984)                                    5.0
Fargo (1996)                                       5.0
Dumb & Dumber (Dumb and Dumber) (1994)             5.0
Dumbo (1941)                                       5.0
Ghost and the Darkness, The (1996)                 5.0
Sword in the Stone, The (1963)                     5.0
Black Cauldron, The (1985)                         5.0
Name: 1, dtype: float64

>10개의 영화만 봐서는 둘의 성향이 비슷하다는 것을 알 수는 없다.

In [13]:
print(type(df_ratings_movies_pivot.loc[1]))
df_ratings_movies_pivot.loc[1]

<class 'pandas.core.series.Series'>


title
'71 (2014)                                   0.0
'Hellboy': The Seeds of Creation (2004)      0.0
'Round Midnight (1986)                       0.0
'Salem's Lot (2004)                          0.0
'Til There Was You (1997)                    0.0
                                            ... 
eXistenZ (1999)                              0.0
xXx (2002)                                   0.0
xXx: State of the Union (2005)               0.0
¡Three Amigos! (1986)                        4.0
À nous la liberté (Freedom for Us) (1931)    0.0
Name: 1, Length: 9719, dtype: float64

###3.1.1 Quiz
> 위의 코드를 dataframe으로 변환하여 출력하세요

In [14]:
def get_score_movie(data_frame, userId):
  # userId에 따른 영화 추철
  user_movie = data_frame.loc[userId]
  # print(user_movie)
  #dataframe으로 변환
  # movie_list = [user_movie.index, user_movie.values]
  # df_movies_score = pd.DataFrame(data=movie_list)
  # df_movies_score.index=['title', 'score']
  # return df_movies_score.transpose()

  # dict을 이용한 dataframe 생성
  # df_movies_score = pd.DataFrame(data={'title':user_movie.index, 'score':user_movie.values})
  # return df_movies_score

  # series를 dataframe으로 변환
  df_movies_score = user_movie.to_frame(name='score')
  return df_movies_score.reset_index()

get_score_movie(df_ratings_movies_pivot, 1)

,title,score
0,'71 (2014),0.0
1,'Hellboy': The Seeds of Creation (2004),0.0
2,'Round Midnight (1986),0.0
3,'Salem's Lot (2004),0.0
4,'Til There Was You (1997),0.0
...,...,...
9714,eXistenZ (1999),0.0
9715,xXx (2002),0.0
9716,xXx: State of the Union (2005),0.0
9717,¡Three Amigos! (1986),4.0


In [15]:
# user1의 모든 영화 목록
df_user1_movies = get_score_movie(df_ratings_movies_pivot, 1)
df_user1_movies

# user266의 모든 영화 목록
df_user266_movies = get_score_movie(df_ratings_movies_pivot, 266)
df_user266_movies

,title,score
0,'71 (2014),0.0
1,'Hellboy': The Seeds of Creation (2004),0.0
2,'Round Midnight (1986),0.0
3,'Salem's Lot (2004),0.0
4,'Til There Was You (1997),0.0
...,...,...
9714,eXistenZ (1999),0.0
9715,xXx (2002),0.0
9716,xXx: State of the Union (2005),0.0
9717,¡Three Amigos! (1986),0.0


###3.1.2 Quiz
> user1이 안 본 영화 중 user266이 본 4.5 이상의 상위10개 영화를 추천하세요

In [16]:
# user1이 안 본 영화 목록
df_user1_movies_0 = df_user1_movies[df_user1_movies['score']==0]
# user266이 본 4.5 이상의 영화 목록
df_user266_movies_5 = df_user266_movies[df_user266_movies['score']>4]

# user 266이 4.5이상 본 영화 중 user1이 안 본 영화
movie_list=[]
for movie_name in df_user266_movies_5['title']:
  df_recommand_movie = df_user1_movies_0[df_user1_movies_0['title']==movie_name]
  lst = df_recommand_movie['title'].tolist()
  # lst가 존재하면
  if lst:
    # print(lst)
    movie_list.append(lst)

print(len(movie_list))
for i in range(10):
  print(movie_list[i])

35
['2001: A Space Odyssey (1968)']
['Aliens (1986)']
['Austin Powers: The Spy Who Shagged Me (1999)']
['Blade Runner (1982)']
['Brazil (1985)']
['Casino (1995)']
['Commitments, The (1991)']
['Crow, The (1994)']
['Dirty Work (1998)']
['Donnie Brasco (1997)']


##3.2 다수의 사용자 기반 추천
> 1. 유사도가 높은 사용자 추출
2. 추천 받을 사람의 안본 영화 추출
3. 추천하는 사람의 4점 이상 영화 추출
4. 2, 3번을 이용한 추천 영화 목록 만들기
5. 2 ~ 4 반복하여 모든 사람의 영화 목록 정보 저장
6. 빈도수 구하기
7. 빈도수가 높은 영화 추천

###3.2.1 유사도가 높은 사용자 추출

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

TARGET_USER = 1

def get_user_similarity(data_frame, userId, top_N=10):
  user_sim = cosine_similarity(data_frame, data_frame)
  df_user_sim = pd.DataFrame(data=user_sim, index=data_frame.index, columns=data_frame.index)
  return df_user_sim[userId].sort_values(ascending=False)[1:top_N+1]

get_user_similarity(df_ratings_movies_pivot, TARGET_USER)

userId
266    0.357408
313    0.351562
368    0.345127
57     0.345034
91     0.334727
469    0.330664
39     0.329782
288    0.329700
452    0.328048
45     0.327922
Name: 1, dtype: float64

> 가독성을 높이기위해 TARGET_USER = 1과 같이 숫자에 의미를 부여하였다.

###3.2.2 추천 받을 사람의 안본 영화 추출

In [18]:
def get_score_movie(data_frame, userId):
  user_movie = data_frame.loc[userId]
  df_movies_score = user_movie.to_frame(name='score')
  df_movies_score = df_movies_score.reset_index()

  return df_movies_score[df_movies_score['score']==0]

get_score_movie(df_ratings_movies_pivot, TARGET_USER).describe()

,score
count,9487.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


###3.2.3 추천하는 사람의 4점 이상 영화 추출

In [19]:
def get_score_movie(data_frame, userId):
  user_movie = data_frame.loc[userId]
  df_movies_score = user_movie.to_frame(name='score')
  df_movies_score = df_movies_score.reset_index()

  return df_movies_score[df_movies_score['score']>=4]

get_score_movie(df_ratings_movies_pivot, TARGET_USER).describe()

,score
count,200.000000
mean,4.620000
std,0.486604
min,4.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [20]:
# 안 본 영화
NON_MOVIE = 0
# 평가 점수
HIGH_SCORE = 4

def get_score_movie(data_frame, userId, score=HIGH_SCORE):
  # userId에 따른 영화 목록을 dataframe으로 변환
  user_movie = data_frame.loc[userId]
  df_movies_score = user_movie.to_frame(name='score')
  df_movies_score = df_movies_score.reset_index()

  # 만약 안 본 영화이면
  if score == NON_MOVIE:
    return df_movies_score[df_movies_score['score']==NON_MOVIE]
  # 평가가 score이상인 영화이면
  return df_movies_score[df_movies_score['score']>=score]

get_score_movie(df_ratings_movies_pivot, TARGET_USER).describe()

,score
count,200.000000
mean,4.620000
std,0.486604
min,4.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


###3.2.4 2, 3번을 이용한 추천 영화 목록 만들기

In [21]:
def get_min_max(df_non_movies, df_highscore_movies):
  # 두 dataframe을 비교해여 짧은 길이의 dataframe을 앞에서 
  # 긴 dataframe을 뒤에서 반환
  if len(df_non_movies) > len(df_highscore_movies):
    return df_highscore_movies, df_non_movies
  return df_non_movie, df_highscore_movies

def get_recommand_movies(df_non_movies, df_highscore_movies):
  # 반복문을 최소화 하기 위해 적은 영화를 가지고 있는 dataframe 얻기
  df_min_movie, df_max_movie = get_min_max(df_non_movies, df_highscore_movies)

  df_recommand_movies = pd.DataFrame()

  for movie_name in df_min_movie['title']:
    df_movie_title = df_max_movie[df_max_movie['title']==movie_name]
  
    if not df_movie_title.empty:
      # print(df_movie_title)
      df_recommand_movies = df_recommand_movies.append(df_movie_title['title'].to_frame())

  return df_recommand_movies

USER1 = 1
USER2 = 266

df_non_movies = get_score_movie(df_ratings_movies_pivot, USER1, NON_MOVIE)
df_highscore_movies = get_score_movie(df_ratings_movies_pivot, USER2, HIGH_SCORE)
get_recommand_movies(df_non_movies, df_highscore_movies)

,title
74,2001: A Space Odyssey (1968)
333,Aliens (1986)
470,Analyze This (1999)
675,Austin Powers: The Spy Who Shagged Me (1999)
1144,Blade (1998)
1146,Blade Runner (1982)
1338,Brazil (1985)
1598,Casino (1995)
1759,"Christmas Story, A (1983)"
1915,"Commitments, The (1991)"


In [22]:
# 추천 받을 영화 수
RECOMMAND_MOVIE = 10
# 추천 받을 사용자
TARGET_USER = 1
# 안본 영화
NON_MOVIE = 0
# 추천 받을 최저점
HIGH_SCORE = 4


# 유사도가 높은 사용자 추출
user_sim = get_user_similarity(df_ratings_movies_pivot, TARGET_USER)
# 추천 받을 사람의 안본 영화 추출
df_non_movies = get_score_movie(df_ratings_movies_pivot, TARGET_USER, NON_MOVIE)

# 추천 받을 영화 저장소
df_recommand_movies = pd.DataFrame()

# 2 ~ 4 반복하여 모든 사람의 영화 목록 정보 저장
for userId in user_sim.index:
  # 추천하는 사람의 4점 이상 영화 추출
  df_highscore_movies = get_score_movie(df_ratings_movies_pivot, userId, HIGH_SCORE)
  # 추천 영화 목록 만들기
  df_recommand_movie = get_recommand_movies(df_non_movies, df_highscore_movies)
  df_recommand_movies = df_recommand_movies.append(df_recommand_movie)


# 빈도수 구하기
df_class_movies = df_recommand_movies.groupby('title')
# 빈도수가 높은 영화 추천
df_class_movies['title'].count().sort_values(ascending=False)[:RECOMMAND_MOVIE]

title
Terminator 2: Judgment Day (1991)            9
Aliens (1986)                                8
Godfather, The (1972)                        8
Godfather: Part II, The (1974)               8
Sixth Sense, The (1999)                      8
Die Hard (1988)                              8
Blade Runner (1982)                          7
Gattaca (1997)                               7
Hunt for Red October, The (1990)             7
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)    7
Name: title, dtype: int64

###3.2.5 가중치
> 유사도와 평점을 곱하여 새로운 점수를 만들고 이를 기준으로 추천

In [ ]:
def get_recommand_movies(df_non_movies, df_highscore_movies, user_similarity):
  # 반복문을 최소화 하기 위해 적은 영화를 가지고 있는 dataframe 얻기
  df_min_movie, df_max_movie = get_min_max(df_non_movies, df_highscore_movies)

  df_recommand_movies = pd.DataFrame(columns=['title', 'score'])

  # for movie_name in df_min_movie['title']:
  # zip함수에 인자로 전달된 두 값을 반환
  for title, score in zip(df_min_movie['title'], df_min_movie['score']):
    df_movie_title = df_max_movie[df_max_movie['title']==title].copy()
    # df_movie_title = df_max_movie[df_max_movie['title']==title]
    
    if not df_movie_title.empty:
      # score가 df_non_movies인 경우 0을 갖게 된다.
      if score == NON_MOVIE:
        # 따라서 df_highscore_movies의 score 정보로 변경한다.
        score = df_movie_title['score']

      df_movie_title['score'] = score * user_similarity
      # df_recommand_movies = df_recommand_movies.append(df_movie_title['title'].to_frame())
      df_recommand_movies = df_recommand_movies.append(df_movie_title)

  return df_recommand_movies


USER1 = 1
USER2 = 266

df_non_movies = get_score_movie(df_ratings_movies_pivot, USER1, NON_MOVIE)
df_highscore_movies = get_score_movie(df_ratings_movies_pivot, USER2, HIGH_SCORE)

get_recommand_movies(df_non_movies, df_highscore_movies, user_sim.values[0])

> 아래와 같은 경고가 나타나는 것은 데이터 저장 시 이전 데이터가 변경될 수 있어 copy를 적용해야 한다.
* /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead
* See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [ ]:
RECOMMAND_MOVIE = 10
TARGET_USER = 1
NON_MOVIE = 0
HIGH_SCORE = 4

user_sim = get_user_similarity(df_ratings_movies_pivot, TARGET_USER)
df_non_movies = get_score_movie(df_ratings_movies_pivot, TARGET_USER, NON_MOVIE)

df_recommand_movies = pd.DataFrame()

# for userId in user_sim.index:
for userId, usr_sim in zip(user_sim.index, user_sim.values):
  df_highscore_movies = get_score_movie(df_ratings_movies_pivot, userId, HIGH_SCORE)
  df_recommand_movie = get_recommand_movies(df_non_movies, df_highscore_movies, usr_sim)
  df_recommand_movies = df_recommand_movies.append(df_recommand_movie)

df_class_movies = df_recommand_movies.groupby('title')
print( df_class_movies['score'].count().sort_values(ascending=False)[:RECOMMAND_MOVIE] )
print( df_class_movies['score'].sum().sort_values(ascending=False)[:RECOMMAND_MOVIE] )

##3.3 아이템 기반 추천

###3.3.1 영화별 유사도 측정

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

df_ratings_movies_pivot_T = df_ratings_movies_pivot.transpose()

movies_sim = cosine_similarity(df_ratings_movies_pivot_T, df_ratings_movies_pivot_T)
movies_sim

In [ ]:
import pandas as pd

df_movies_sim = pd.DataFrame(data=movies_sim, index=df_ratings_movies_pivot.columns, columns=df_ratings_movies_pivot.columns)
df_movies_sim

In [ ]:
df_movies_sim["'Hellboy': The Seeds of Creation (2004)"].sort_values(ascending=False)[:10]

##3.4 개인별 영화 추천

In [33]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_mse(pred, data_frame):
  rating_pred_nonzero = pred[data_frame.nonzero()]
  df_rating_nonzero = data_frame[data_frame.nonzero()]

  return mean_squared_error(rating_pred_nonzero, df_rating_nonzero)

def predict_rating_top(ratings_arr, item_sim, top=10):
  pred = np.zeros(ratings_arr.shape)

  for row in range(ratings_arr.shape[0]):
    top_n_item = np.argsort(ratings_arr[row, :])[::-1][:top]
    for col in range(ratings_arr.shape[1]):
      pred[row, col] = item_sim[:, col][top_n_item].dot(ratings_arr[row, :][top_n_item])
      pred[row, col] /= np.sum(np.abs(item_sim[col, :][top_n_item]))
  
  return pred

ratings_pred = predict_rating_top(df_ratings_movies_pivot.values, movies_sim)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


In [35]:
print(get_mse(ratings_pred, df_ratings_movies_pivot.values))

3.0918230908381696


In [ ]:
df_ratings_pred = pd.DataFrame(data=ratings_pred, index=df_ratings_movies_pivot.index, columns=df_ratings_movies_pivot.columns)
df_ratings_pred

In [39]:
def get_unseen_movies(data_frame, userId):
  # userid에 설정된 모든 영화 추출
  user_rating = data_frame.loc[userId]
  # 안본 영화 추출
  unseen_list = user_rating[user_rating==0].index
  # 리스트화
  return unseen_list.tolist()

unseen_list = get_unseen_movies(df_ratings_movies_pivot, 1)

In [41]:
def recomm_by_user(pred_df, userId, unseen_list, top_n=10):
  # 사용자의 예측 점수가 포함된 영화
  movies = pred_df.loc[userId]
  # 안본 영화 목록
  unseen_movies = movies[unseen_list]
  # 상위 top n
  recomm_movies = unseen_movies.sort_values(ascending=False)[:top_n]
  return recomm_movies

recomm_by_user(df_ratings_pred, 1, unseen_list, 10)

title
National Lampoon's Van Wilder (2002)                  5.0
Lethal Weapon 3 (1992)                                5.0
Cider House Rules, The (1999)                         5.0
Sorry, Wrong Number (1948)                            5.0
Ant-Man (2015)                                        5.0
Showtime (2002)                                       5.0
NeverEnding Story II: The Next Chapter, The (1990)    5.0
Piano, The (1993)                                     5.0
Darkest Hour (2017)                                   5.0
Titanic (1997)                                        5.0
Name: 1, dtype: float64